In [1]:
using Pkg
Pkg.activate("."); Pkg.instantiate()
Pkg.precompile()                                                                

using CSV                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
using DataFrames
using Plots
using Glob
using Dates
using Distances: haversine

  Activating project at `~/proyectos/TFG-ProduccionElectrica/codigo/Julia`


In [2]:
filepath = "../data/predict-energy-behavior-of-prosumers"
files = glob("*.csv", filepath)

8-element Vector{String}:
 "../data/predict-energy-behavior-of-prosumers/client.csv"
 "../data/predict-energy-behavior-of-prosumers/county_lon_lats.csv"
 "../data/predict-energy-behavior-of-prosumers/electricity_prices.csv"
 "../data/predict-energy-behavior-of-prosumers/forecast_weather.csv"
 "../data/predict-energy-behavior-of-prosumers/gas_prices.csv"
 "../data/predict-energy-behavior-of-prosumers/historical_weather.csv"
 "../data/predict-energy-behavior-of-prosumers/train.csv"
 "../data/predict-energy-behavior" ⋯ 20 bytes ⋯ "r_station_to_county_mapping.csv"

In [3]:
dfs = Dict(
    splitext(basename(archivo))[1] => CSV.read(archivo, DataFrame)
    for archivo in files
)
dfs["weather_station"] = dfs["weather_station_to_county_mapping"]
delete!(dfs, "weather_station_to_county_mapping")

Dict{String, DataFrame} with 8 entries:
  "historical_weather" => 1710802×18 DataFrame…
  "electricity_prices" => 15286×4 DataFrame…
  "weather_station"    => 112×4 DataFrame…
  "client"             => 41919×7 DataFrame…
  "train"              => 2018352×9 DataFrame…
  "forecast_weather"   => 3424512×18 DataFrame…
  "county_lon_lats"    => 75×4 DataFrame…
  "gas_prices"         => 637×5 DataFrame…

In [71]:
hist_w = select(dfs["historical_weather"], [:latitude, :longitude])
hits_w_un = unique(hist_w, [:latitude, :longitude])
stations = dropmissing(
    select(dfs["weather_station"], Not("county_name"))
)
stations = select(
    stations,
    :latitude => ByRow(deg2rad) => :latitude,
    :longitude => ByRow(deg2rad) => :longitude,
    :county
)

Row,latitude,longitude,county
,Float64,Float64,Int64
1,1.01578,0.387463,10
2,1.02102,0.387463,10
3,1.02102,0.39619,10
4,1.02625,0.39619,1
5,1.02102,0.404916,10
6,1.02102,0.413643,7
7,1.02625,0.413643,6
8,1.03149,0.413643,6
9,1.02102,0.42237,7


In [72]:
hits_w_un = transform(
    hits_w_un,
    :latitude => ByRow(deg2rad) => :lat_rad,
    :longitude => ByRow(deg2rad) => :lon_rad
)

Row,latitude,longitude,lat_rad,lon_rad
,Float64,Float64,Float64,Float64
1,57.6,21.7,1.00531,0.378736
2,57.6,22.2,1.00531,0.387463
3,57.6,22.7,1.00531,0.39619
4,57.6,23.2,1.00531,0.404916
5,57.6,23.7,1.00531,0.413643
6,57.6,24.2,1.00531,0.42237
7,57.6,24.7,1.00531,0.431096
8,57.6,25.2,1.00531,0.439823
9,57.6,25.7,1.00531,0.44855


In [54]:
rw = stations[1, :]

Row,latitude,longitude,county
,Float64,Float64,Int64
1,1.01578,0.387463,10


In [56]:
rw[[:latitude, :longitude]]

Row,latitude,longitude
,Float64,Float64
1,1.01578,0.387463


In [57]:
a = zeros(Int, 0)
append!(a, 4)
append!(a, 54)

2-element Vector{Int64}:
  4
 54

In [73]:
counties = zeros(Int, 0)

for coord in eachrow(hits_w_un[:, 3:4])
    min_dist = Inf
    cnty = -1

    for station in eachrow(stations)
        dist = haversine(coord, 
        [station.latitude, station.longitude])

        if dist < min_dist
            cnty = station[:county]
            min_dist = dist
        end
    end

    append!(counties, cnty)
end
counties


112-element Vector{Int64}:
 10
 10
 10
 10
  7
  7
  7
  7
 13
 13
  ⋮
  0
  0
  0
  0
  5
  5
  2
  2
  2

In [87]:
hits_w_un[!, :county] = counties
hits_w_un
hw_cp = select(dfs["historical_weather"], All())

Row,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
,String31,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.08333,8,0.0,0.0,0.0,57.6,21.7,1.0
2,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.11111,359,0.0,0.0,0.0,57.6,22.2,1.0
3,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.33333,355,0.0,0.0,0.0,57.6,22.7,1.0
4,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.08333,297,358.0,277.0,81.0,57.6,23.2,1.0
5,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.41667,5,0.0,0.0,0.0,57.6,23.7,1.0
6,2021-09-01 00:00:00,16.0,11.4,0.0,0.0,1016.7,15,12,7,0,6.38889,297,349.0,274.0,75.0,57.6,24.2,1.0
7,2021-09-01 00:00:00,12.2,10.5,0.0,0.0,1005.3,19,7,0,41,4.11111,359,0.0,0.0,0.0,57.6,24.7,1.0
8,2021-09-01 00:00:00,12.2,10.5,0.0,0.0,1006.1,28,8,0,68,4.11111,354,0.0,0.0,0.0,57.6,25.2,1.0
9,2021-09-01 00:00:00,12.4,10.6,0.0,0.0,1003.8,49,26,0,85,4.25,352,0.0,0.0,0.0,57.6,25.7,1.0


In [89]:
leftjoin(hw_cp, hits_w_un, on=["latitude", "longitude"])

Row,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,lat_rad,lon_rad,county
,String31,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Float64?,Int64?
1,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.08333,8,0.0,0.0,0.0,57.6,21.7,1.0,1.00531,0.378736,10
2,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.11111,359,0.0,0.0,0.0,57.6,22.2,1.0,1.00531,0.387463,10
3,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.33333,355,0.0,0.0,0.0,57.6,22.7,1.0,1.00531,0.39619,10
4,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.08333,297,358.0,277.0,81.0,57.6,23.2,1.0,1.00531,0.404916,10
5,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.41667,5,0.0,0.0,0.0,57.6,23.7,1.0,1.00531,0.413643,7
6,2021-09-01 00:00:00,16.0,11.4,0.0,0.0,1016.7,15,12,7,0,6.38889,297,349.0,274.0,75.0,57.6,24.2,1.0,1.00531,0.42237,7
7,2021-09-01 00:00:00,12.2,10.5,0.0,0.0,1005.3,19,7,0,41,4.11111,359,0.0,0.0,0.0,57.6,24.7,1.0,1.00531,0.431096,7
8,2021-09-01 00:00:00,12.2,10.5,0.0,0.0,1006.1,28,8,0,68,4.11111,354,0.0,0.0,0.0,57.6,25.2,1.0,1.00531,0.439823,7
9,2021-09-01 00:00:00,12.4,10.6,0.0,0.0,1003.8,49,26,0,85,4.25,352,0.0,0.0,0.0,57.6,25.7,1.0,1.00531,0.44855,13


In [100]:
struct DataProcessor
    gas_join::Vector{String}
    ep_join::Vector{String}
    hw_join::Vector{String}
    client_join::Vector{String}
    lat_lon::Vector{String}
end

function DataProcessor()
    DataProcessor(
        ["datetime"],
        ["datetime"],
        ["county", "datetime"],
        ["product_type", "county", "is_business", "date"],
        ["latitude", "longitude"]
    )
end

function change_names(df::DataFrame, suffix::AbstractString, no_change::Vector{String})
    renamed_cols = [
        col in no_change ? col : col * suffix for col in names(df)
    ]

    renamed_df = rename(df, names(df) .=> renamed_cols)
    return renamed_df
end

function to_datetime(df::DataFrame, col::AbstractString)
    new_df = deepcopy(df)
    formato = DateFormat("yyyy-mm-dd HH:MM:SS")

    new_df[!, col] .= DateTime.(new_df[!, col], formato)
    return new_df
end

function get_client_features(client_df::DataFrame, client_join::Vector{String})
    return change_names(client_df, "_client", client_join)
end

function get_gas_features(gas_df::DataFrame, gas_join::Vector{String})
    df = deepcopy(gas_df)

    df[!, "mean"] = (df.highest_price_per_mwh + df.lowest_price_per_mwh) / 2

    df = change_names(df, "_gas", gas_join)

    return df
end

function get_electricity_features(ep_df::DataFrame, ep_join::Vector{String})
    df = to_datetime(ep_df, "forecast_date")
    df = rename(df, :forecast_date => :datetime)
    df = change_names(df, "_ep", ep_join)
    return df
end

function get_data_features(data::DataFrame)
    df = to_datetime(data, "datetime")
    df.date = Date.(df.datetime)
    df.year = year.(df.datetime)
    df.quarter = quarterofyear.(df.datetime)
    df.month = month.(df.datetime)
    df.week = week.(df.datetime)
    df.hour = hour.(df.datetime)
    df.day_of_year = dayofyear.(df.datetime)
    df.day_of_month = dayofmonth.(df.datetime)
    df.day_of_week = dayofweek.(df.datetime)

    return df
end

function get_counties(histw_df:: DataFrame, wst_df:: DataFrame)
    coords = select(histw_df, [:latitude, :longitude])
    coords = unique(coords, [:latitude, :longitude])
    coords = transform(
        coords,
        :latitude => ByRow(deg2rad) => :lat_rad,
        :longitude => ByRow(deg2rad) => :lon_rad
    )

    stations = dropmissing(
        select(
            wst_df,
            :latitude => ByRow(deg2rad) => :latitude,
            :longitude => ByRow(deg2rad) => :longitude,
            :county
        )
    )

    counties = zeros(Int, 0)

    for coord in eachrow(coords[:, [:lat_rad, :lon_rad]])
        min_dist = Inf
        cnty = -1

        for station in eachrow(stations)
            dist = haversine(
                coord,
                [station.latitude, station.longitude]
            )

            if dist < min_dist
                cnty = station.county
                min_dist = dist
            end
        end

        append!(counties, cnty)
    end

    coords[:, :county] = counties

    coords = select(coords, Not([:lat_rad, :lon_rad]))

    histw_join = leftjoin(histw_df, coords, on=["latitude", "longitude"])
    return histw_join
end

function get_hist_weather_features(hw_df::DataFrame, ws_df::DataFrame, hw_joins::Vector{String})
    df = to_datetime(hw_df, "datetime")
    df = get_counties(hw_df, ws_df)
    df = change_names(df, "_hw", hw_joins)
    return df
end

function process_data(dp::DataProcessor, df_dict::Dict)
    data = get_data_features(df_dict["train"])
    client = get_client_features(df_dict["client"], dp.client_join)
    gas = get_gas_features(df_dict["gas_prices"], dp.gas_join)
    electricity = get_electricity_features(df_dict["electricity_prices"], dp.ep_join)
    hist_wthr = get_hist_weather_features(
        df_dict["historical_weather"],
        df_dict["weather_station"],
        dp.hw_join
    )

    joins = [dp.client_join, dp.ep_join, dp.gas_join]
    final_data = leftjoin(data, gas, client, electricity, on=joins)
    #final_data = leftjoin(final_data, hist_wthr, on=dp.hw_join)
    
    return final_data
end

process_data (generic function with 1 method)

In [101]:
processor = DataProcessor()
process_data(processor, dfs)

MethodError: MethodError: no method matching leftjoin(::DataFrame, ::DataFrame, ::DataFrame, ::DataFrame; on::Vector{Vector{String}})
The function `leftjoin` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  leftjoin(::AbstractDataFrame, ::AbstractDataFrame; on, makeunique, source, indicator, validate, renamecols, matchmissing, order)
   @ DataFrames ~/.julia/packages/DataFrames/kcA9R/src/join/composer.jl:917


In [77]:
dfs["historical_weather"]

Row,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
,String31,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,2021-09-01 00:00:00,14.2,11.6,0.0,0.0,1015.9,31,31,0,11,7.08333,8,0.0,0.0,0.0,57.6,21.7,1.0
2,2021-09-01 00:00:00,13.9,11.5,0.0,0.0,1010.7,33,37,0,0,5.11111,359,0.0,0.0,0.0,57.6,22.2,1.0
3,2021-09-01 00:00:00,14.0,12.5,0.0,0.0,1015.0,31,34,0,0,6.33333,355,0.0,0.0,0.0,57.6,22.7,1.0
4,2021-09-01 00:00:00,14.6,11.5,0.0,0.0,1017.3,0,0,0,0,8.08333,297,358.0,277.0,81.0,57.6,23.2,1.0
5,2021-09-01 00:00:00,15.7,12.9,0.0,0.0,1014.0,22,25,0,0,8.41667,5,0.0,0.0,0.0,57.6,23.7,1.0
6,2021-09-01 00:00:00,16.0,11.4,0.0,0.0,1016.7,15,12,7,0,6.38889,297,349.0,274.0,75.0,57.6,24.2,1.0
7,2021-09-01 00:00:00,12.2,10.5,0.0,0.0,1005.3,19,7,0,41,4.11111,359,0.0,0.0,0.0,57.6,24.7,1.0
8,2021-09-01 00:00:00,12.2,10.5,0.0,0.0,1006.1,28,8,0,68,4.11111,354,0.0,0.0,0.0,57.6,25.2,1.0
9,2021-09-01 00:00:00,12.4,10.6,0.0,0.0,1003.8,49,26,0,85,4.25,352,0.0,0.0,0.0,57.6,25.7,1.0


In [70]:
as = rename(
    dfs["electricity_prices"],
    :forecast_date => :datetime
);

formato = DateFormat("yyyy-mm-dd HH:MM:SS");

as.datetime = DateTime.(as.datetime, formato);
as.date = Date.(as.datetime)
as.year = year.(as.datetime)
as.quarter = quarterofyear.(as.datetime)
as.month = month.(as.datetime)
as.day_of_year = dayofyear.(as.datetime)
as.day_of_month = dayofmonth.(as.datetime)
as.day_of_week = dayofweek.(as.datetime)
as.week = week.(as.datetime)
as.hour = hour.(as.datetime)

as

Row,datetime,euros_per_mwh,origin_date,data_block_id,date,year,quarter,month,day_of_year,day_of_month,day_of_week,week,hour
,DateTime,Float64,String31,Int64,Date,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2021-09-01T00:00:00,92.51,2021-08-31 00:00:00,1,2021-09-01,2021,3,9,244,1,3,35,0
2,2021-09-01T01:00:00,88.9,2021-08-31 01:00:00,1,2021-09-01,2021,3,9,244,1,3,35,1
3,2021-09-01T02:00:00,87.35,2021-08-31 02:00:00,1,2021-09-01,2021,3,9,244,1,3,35,2
4,2021-09-01T03:00:00,86.88,2021-08-31 03:00:00,1,2021-09-01,2021,3,9,244,1,3,35,3
5,2021-09-01T04:00:00,88.43,2021-08-31 04:00:00,1,2021-09-01,2021,3,9,244,1,3,35,4
6,2021-09-01T05:00:00,93.58,2021-08-31 05:00:00,1,2021-09-01,2021,3,9,244,1,3,35,5
7,2021-09-01T06:00:00,118.7,2021-08-31 06:00:00,1,2021-09-01,2021,3,9,244,1,3,35,6
8,2021-09-01T07:00:00,135.44,2021-08-31 07:00:00,1,2021-09-01,2021,3,9,244,1,3,35,7
9,2021-09-01T08:00:00,140.72,2021-08-31 08:00:00,1,2021-09-01,2021,3,9,244,1,3,35,8


In [64]:
as2 = copy(dfs["gas_prices"])

Row,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
,DateTime,Float64,Float64,Date,Int64
1,2021-09-01T00:00:00,45.23,46.32,2021-08-31,1
2,2021-09-02T00:00:00,45.62,46.29,2021-09-01,2
3,2021-09-03T00:00:00,45.85,46.4,2021-09-02,3
4,2021-09-04T00:00:00,46.3,46.8,2021-09-03,4
5,2021-09-05T00:00:00,46.3,46.58,2021-09-04,5
6,2021-09-06T00:00:00,46.17,46.95,2021-09-05,6
7,2021-09-07T00:00:00,46.35,47.6,2021-09-06,7
8,2021-09-08T00:00:00,46.4,47.64,2021-09-07,8
9,2021-09-09T00:00:00,44.96,47.72,2021-09-08,9


In [ ]:
as2[!, "mean"] = (as2.lowest_price_per_mwh .+ as2.highest_price_per_mwh) ./ 2

637-element Vector{Float64}:
 45.775
 45.955
 46.125
 46.55
 46.44
 46.56
 46.975
 47.019999999999996
 46.34
 48.055
  ⋮
 37.105
 34.7
 35.260000000000005
 32.5
 31.6
 31.200000000000003
 31.1
 32.57
 31.5

In [66]:
as2

Row,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id,mean
,DateTime,Float64,Float64,Date,Int64,Float64
1,2021-09-01T00:00:00,45.23,46.32,2021-08-31,1,45.775
2,2021-09-02T00:00:00,45.62,46.29,2021-09-01,2,45.955
3,2021-09-03T00:00:00,45.85,46.4,2021-09-02,3,46.125
4,2021-09-04T00:00:00,46.3,46.8,2021-09-03,4,46.55
5,2021-09-05T00:00:00,46.3,46.58,2021-09-04,5,46.44
6,2021-09-06T00:00:00,46.17,46.95,2021-09-05,6,46.56
7,2021-09-07T00:00:00,46.35,47.6,2021-09-06,7,46.975
8,2021-09-08T00:00:00,46.4,47.64,2021-09-07,8,47.02
9,2021-09-09T00:00:00,44.96,47.72,2021-09-08,9,46.34
